# 🥈 Silver Layer - Cleaned and Enriched Data

The **Silver Layer** applies data cleaning, standardization, enrichment, and minor business logic to the raw Bronze Layer tables. This layer ensures consistent, structured, and analytics-ready data.

---


## Import Required Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Step 1: Create Silver Database

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS silver;

# Load Bronze Layer Tables

In [0]:
df_athletes = spark.table("bronze.athletes")
df_coaches = spark.table("bronze.coaches")
df_entriesGender = spark.table("bronze.entriesGender")
df_medals = spark.table("bronze.medals")
df_teams = spark.table("bronze.teams")

### Data Cleaning using PySpark

#### 🔧 Utility Function: `clean_df`

This function handles standard cleaning tasks for Spark DataFrames:

- Drops duplicate records
- Removes rows with nulls in critical columns
- Fills missing values using a dictionary
- Standardizes column names (lowercase, no spaces)

In [0]:
def clean_df(df, critical_columns=[], fill_na_dict={}):
    # Drop duplicates
    df = df.dropDuplicates()

    # Drop rows with nulls in important columns
    if critical_columns:
        df = df.dropna(subset=critical_columns)

    # Fill NA values
    if fill_na_dict:
        df = df.fillna(fill_na_dict)

    # Standardize column names: remove spaces, make lowercase
    for c in df.columns:
        new_col = c.strip().lower().replace(" ", "_")
        df = df.withColumnRenamed(c, new_col)

    return df


####1. df_athletes

In [0]:
cleaned_df_athletes = clean_df(
    df_athletes,
    critical_columns=["PersonName", "Discipline"],
    fill_na_dict={"PersonName": "n/a", "Country": "n/a", "Discipline": "n/a"}
)
cleaned_df_athletes = cleaned_df_athletes.withColumnRenamed("personname", "person_name")


cleaned_df_athletes.write.format("delta").mode("overwrite").saveAsTable("silver.athletes")

####2. df_coaches

In [0]:
cleaned_df_coaches = clean_df(
    df_coaches,
    critical_columns=["Name", "Discipline"],
    fill_na_dict={"Name": "n/a", "Country": "n/a", "Discipline": "n/a", "Event": "n/a"}
)

cleaned_df_coaches.write.format("delta").mode("overwrite").saveAsTable("silver.coaches")

####3. df_entriesGender

In [0]:
cleaned_df_entries_gender = clean_df(
    df_entriesGender,
    critical_columns=["Discipline", "Female", "Male", "Total"],
    fill_na_dict={"Discipline": "n/a", "Female": 0, "Male": 0, "Total": 0}
)
cleaned_df_entries_gender.show()

cleaned_df_entries_gender.write.format("delta").mode("overwrite").saveAsTable("silver.entries_gender")

+--------------------+------+----+-----+
|          discipline|female|male|total|
+--------------------+------+----+-----+
|             Surfing|    20|  20|   40|
| Artistic Gymnastics|    98|  98|  196|
|           Wrestling|    96| 193|  289|
|   Artistic Swimming|   105|   0|  105|
|          Water Polo|   122| 146|  268|
|      Sport Climbing|    20|  20|   40|
|       Skateboarding|    40|  40|   80|
|   Marathon Swimming|    25|  25|   50|
|                Golf|    60|  60|  120|
|                Judo|   192| 201|  393|
|        Cycling Road|    70| 131|  201|
|        Canoe Sprint|   123| 126|  249|
|Cycling Mountain ...|    38|  38|   76|
|           Athletics|   969|1072| 2041|
|              Hockey|   192| 192|  384|
|              Diving|    72|  71|  143|
|            Swimming|   361| 418|  779|
|            Football|   264| 344|  608|
|  Cycling BMX Racing|    24|  24|   48|
|           Taekwondo|    65|  65|  130|
+--------------------+------+----+-----+
only showing top

####4. df_medals

In [0]:
cleaned_df_medals = clean_df(
    df_medals,
    critical_columns=["TeamCountry", "Gold", "Silver", "Bronze", "Total", "Rank_by_Total"],
    fill_na_dict={
        "TeamCountry": "n/a",
        "Gold": 0, "Silver": 0, "Bronze": 0, "Total": 0,
        "Rank": 0, "Rank_by_Total": 0
    }
)
cleaned_df_medals = cleaned_df_medals.withColumnRenamed("teamcountry", "team_country")

cleaned_df_medals.write.format("delta").mode("overwrite").saveAsTable("silver.medals")


####5. df_teams

In [0]:
cleaned_df_teams = clean_df(
    df_teams,
    critical_columns=[],
    fill_na_dict={"TeamName": "n/a", "Discipline": "n/a", "Country": "n/a", "Event": "n/a"}
)
cleaned_df_teams = cleaned_df_teams.withColumnRenamed("teamname", "team_name")

cleaned_df_teams.write.format("delta").mode("overwrite").saveAsTable("silver.teams")


## ✅ Output

All cleaned and transformed tables are saved as **Delta Tables** under the `silver` schema for use in the Gold Layer.